In [2]:
import pandas as pd
import numpy as np
import os
import requests
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import random
import time

In [20]:
url1 = 'https://fantasy.premierleague.com/api/entry/4306792/'
r1 = requests.get(url1)
userinfo = r1.json()
userinfo.keys()

dict_keys(['id', 'joined_time', 'started_event', 'favourite_team', 'player_first_name', 'player_last_name', 'player_region_id', 'player_region_name', 'player_region_iso_code_short', 'player_region_iso_code_long', 'summary_overall_points', 'summary_overall_rank', 'summary_event_points', 'summary_event_rank', 'current_event', 'leagues', 'name', 'name_change_blocked', 'kit', 'last_deadline_bank', 'last_deadline_value', 'last_deadline_total_transfers'])

In [59]:
userinfo['id']
userinfo['player_region_name']
userinfo['summary_overall_points']
userinfo['summary_overall_rank']
userinfo['last_deadline_total_transfers']

27

The idea is to find the distribution of FPL users across the world. But to do so, extracting the data from all the users might be limited from the API console and is not safe for my IP. 

One idea is to leverage the statistical sampling methods to estimate those.

Let's start with 50 samples

In [34]:
random.seed(10)
int(np.random.uniform(1,8000000))

5266650

#### First attempt to pull 50 data points. 

In [35]:
id_list = []

for i in range(50):
    id_list.append(int(np.random.uniform(1,8000000)))

In [40]:
len(id_list)

50

In [47]:
data_df = pd.DataFrame(index=range(50), columns=['user_id','user_region','favourite_team',
                                                 'overall_points','overall_rank'])

In [48]:
for i in range(50):
    url2 = 'https://fantasy.premierleague.com/api/entry/' + str(id_list[i]) + '/'
    print(i)
    r2 = requests.get(url2)
    userinfo = r2.json()
    
    data_df.iloc[i,0] = userinfo['id']
    data_df.iloc[i,1] = userinfo['player_region_name']
    data_df.iloc[i,2] = userinfo['favourite_team']
    data_df.iloc[i,3] = userinfo['summary_overall_points']
    data_df.iloc[i,4] = userinfo['summary_overall_rank']

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [49]:
data_df

,user_id,user_region,favourite_team,overall_points,overall_rank
0,7000956,Egypt,13,1010,4639122
1,6080218,Tanzania,6,1011,4624509
2,2293355,England,1,1009,4655681
3,1171088,England,7,1109,2888044
4,611576,England,None,960,5458899
5,1836322,Bangladesh,13,1026,4359575
6,781223,England,13,951,5602363
7,1384092,USA,10,1049,3956853
8,4647824,Wales,9,745,7778044
9,6863627,Jordan,12,859,6834047


In [54]:
data_df.groupby('user_region').count()['user_id']

user_region
Australia                2
Bangladesh               1
Canada                   1
Egypt                    5
England                 11
Hong Kong                1
Iraq                     1
Ireland                  4
Jordan                   1
Kenya                    1
Malaysia                 2
Netherlands              1
New Zealand              1
Nigeria                  2
Northern Ireland         1
Norway                   1
Qatar                    1
Scotland                 3
South Korea              1
Sudan                    2
Sweden                   1
Tanzania                 2
Tunisia                  1
USA                      1
United Arab Emirates     1
Wales                    1
Name: user_id, dtype: int64

#### Scaling up the program to pull 1000 users

In [3]:
id_list_2 = []

for i in range(1000):
    id_list_2.append(int(np.random.uniform(1,8000000)))

In [4]:
data_df_1 = pd.DataFrame(index=range(1000), columns=['user_id','user_region',
                                                     'favourite_team',
                                                     'overall_points',
                                                     'overall_rank',
                                                     'last_deadline_value',
                                                     'last_deadline_total_transfers'])

In [5]:
for i in range(1000):
    url2 = 'https://fantasy.premierleague.com/api/entry/' + str(id_list_2[i]) + '/'
    print(i)
    r2 = requests.get(url2)
    userinfo = r2.json()
    
    data_df_1.iloc[i,0] = userinfo['id']
    data_df_1.iloc[i,1] = userinfo['player_region_name']
    data_df_1.iloc[i,2] = userinfo['favourite_team']
    data_df_1.iloc[i,3] = userinfo['summary_overall_points']
    data_df_1.iloc[i,4] = userinfo['summary_overall_rank']
    data_df_1.iloc[i,5] = userinfo['last_deadline_value']
    data_df_1.iloc[i,6] = userinfo['last_deadline_total_transfers']
    
    time.sleep(1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

ConnectionError: HTTPSConnectionPool(host='fantasy.premierleague.com', port=443): Max retries exceeded with url: /api/entry/2856267/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd27d962580>: Failed to establish a new connection: [Errno 50] Network is down'))

In [6]:
data_df_1.shape

(1000, 7)

In [7]:
data_df_1.to_excel('FPL_users_1k_v01222022_v1.xlsx')